## NMF解析
### main~Correlation
1. 各被験者のファイルの名前リストを受け渡し，NMF_analysisを呼び出す

### NMF_analysis
1. 前処理後のデータ（Preprocessed_bind）を読み込む
1. NMFによりCoef（係数行列;W）とSynergy（基底;H）に分解
1. 各試行データの各ランクに対して，基底/係数行列をcsv出力
1. ランクの増加に伴うVAFの推移をグラフ/csv出力
1. Overall_VAF>0.9, muscle_VAFの**最小値**>0.75を満たす最小のシナジーを求め，グラフ出力
1. 1strideにおける係数行列の平均$\pm$SDを計算/グラフ出力（ここはまだ未完成）

### Correlation
1. 歩行とスラックラインの各試行に対して，上で求めた最適な筋シナジー同士の相関係数を導出し，ヒートマップ/csv出力

## 注意
- ファイルはPreprocessed_bindから読み込む
- **ID5**: **ADD**の筋電位が明らかにおかしいので除いて解析

In [1]:
library(readr)
library(Rmisc) #tidtverseより先に読み込む
library(tidyverse)
library(signal)
library(magrittr)
library(grid) 
library(NMF)
library(dplyr)
library(ggplot2)
library(cowplot) 
library(corrplot) #相関係数のplotに使用
library(squash)   #plot時の色指定に使用
library(maptools)
library(gplots)

ERROR: Error in library(readr): there is no package called ‘readr’


In [2]:
#筋シナジーや係数行列（mean±SD）のmultiplot
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL,graph_text=NULL) {
    plots <- c(list(...), plotlist) # Make a list from the ... arguments and plotlist
    numPlots = length(plots)
    if (is.null(layout)) {  # If layout is NULL, then use 'cols' to determine layout
        layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
    }
    if (numPlots==1){
        print(plots[[1]])
    }else{
        grid.newpage() # Set up the page
        par(oma = c(0, 0, 4, 0))   # 下・左・上・右の順で余白を設定
        pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))
        for (i in 1:numPlots) {  # Make each plot, in the correct location
            matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE)) # Get the i,j matrix positions of the regions that contain this subplot
            print(plots[[i]], vp=viewport(layout.pos.row=matchidx$row, layout.pos.col=matchidx$col))
        }
    }       
}

In [3]:
#labelに該当するphaseの連続回数（1strideのstep数）を求める
get_phase_tlist <- function(fdata_walk_labeled,label){
    fdata_walk_labeled <- as.data.frame(fdata_walk_labeled)
    phase <- fdata_walk_labeled[,ncol(fdata_walk_labeled)]
    count <- 0
    phase_tlist <- c()
    phase_tlist_sum <- c()
    for(i in 1:length(phase)){
        if(phase[i]==-1&&(phase[i]==0||phase[i]==1)&&(phase[i]==0||phase[i]==1)){ #途中(おかしな場所)でlabelが-1であれば，その前までのphase_tlistを返す 
            print("Warning! The data contains invalid label.") 
            break
        }
        if(phase[i]==label){
            count <- count+1
        }else if(i>1&&(phase[i-1]==label)&&(phase[i]!=label)){ #立脚期/または遊脚期が終わってからやっと長さ情報追加＝途中で終わった分は追加されない
            phase_tlist <- append(phase_tlist,count)
            phase_tlist_sum <- cumsum(phase_tlist) #累積和を計算
        }else{
            count <- 0
        }
    }

    return(list(phase_tlist=phase_tlist, phase_tlist_sum=phase_tlist_sum))
}

In [4]:
#1strideにおける係数行列のmean±SDを計算/csv,グラフ出力
Coef_mean_SD <- function(Coef,phase_tlist,phase_tlist_sum,gname_Coef_MEAN_SD,label,id,rand){
    #保存先の作成
    Syn_num <- ncol(Coef)
    ifelse(label==0, x_axis <- "Stance phase", x_axis <- "Swing phase") 
    plots <- list()
    
    for (i in 1:Syn_num){ #各シナジーに対する処理
        for (j in 2:length(phase_tlist)){ # 各strideに対する処理， 始めのstride（最初の1周期）は無視して計算
            time <- 0:(phase_tlist[j]-1)
            norm_time <- sapply(time,function(x) {return(x/max(time))}) #正規化して周期長を0から1に揃える
            onecycle <- (phase_tlist_sum[j-1]+1):phase_tlist_sum[j]
            onecycle_Coef <- Coef[onecycle,i]
            sp <- splinefun(norm_time,onecycle_Coef) #関数spを返す 
            xest <- seq(0,1,by=0.01) #新しい時間軸
            yest <- sp(xest) #新しい時間軸に対するCoefの値

            if (j==2){
                Spline_list <- cbind(xest,yest)
            }else{
                Spline_list <- bind_cols(as.data.frame(Spline_list), as.data.frame(yest))
            }
        }
        
        #平均値, SDを計算    
        Stride <- paste("Stride",1:(length(phase_tlist)-1),sep="")
        colnames(Spline_list) <- append(Stride,"time",after=0)
        mean <- apply(Spline_list[,2:length(phase_tlist)], MARGIN=1, mean) #各行（時間）に対して平均値を計算
        sd <- apply(Spline_list[,2:length(phase_tlist)], MARGIN=1, sd)
        
        #平均値,SDのggplot作成
        if(i==1){
            plot_title_Coef <- paste("Coef",i,"   ",id,"_",rand,sep="")
        }else{
            plot_title_Coef <- paste("Coef",i,sep="")
        }
        Coef_data <- data.frame(stance_phase=xest, mean=mean, sd=sd)
        Coef_plot <- ggplot(data=Coef_data,)+
                        geom_line(aes(x=stance_phase,y=mean))+
                        geom_line(aes(x=stance_phase,y=mean+sd),colour="gray57")+
                        geom_line(aes(x=stance_phase,y=mean-sd),colour="gray57")+
                        xlab(x_axis) + ylab("Activity of Synergy")+
                        labs(title=plot_title_Coef)+ 
                        theme(axis.title=element_text(size=3,face="plain"))+
                        theme_minimal() #背景の色変更
        plots[[i]] <- Coef_plot
    }

    #平均値，SDのmultiplot
    gname <- gname_Coef_MEAN_SD
    pdf(gname) 
    multiplot(plotlist=plots,cols=2)
    dev.off() 
}

In [5]:
NMF_analysis <- function(header,file_name,muscle_num,id,label,task,rand){
    #データの読み込み
    print(file_name)
    if(task=="walk"){
        fdata_walk_labeled <- read_csv(file_name,col_types="dddddddddddddddddddi")%>%
                                set_colnames(header)
        res <- get_phase_tlist(fdata_walk_labeled,label) #1strideごとのstep数を取得
        phase_tlist <- res$phase_tlist 
        phase_tlist_sum <- res$phase_tlist_sum
        Preprocessed <- fdata_walk_labeled%>%
                        dplyr::filter(phase==label)%>%
                        select(-c(time,y_pos,x_pos,phase))
    }else{
        Preprocessed <- read_csv(file_name,col_types="ddddddddddddddddddd")%>%
                        set_colnames(header)%>%
                        select(-c(time,ankle_X,ankle_Z)) 
    }
    
    if(id=="ID3"){ 
        Preprocessed <- Preprocessed%>%select(-c(SOL,EO)) #ID3からはSOLとEOを除く
    }else if(id=="ID5"){
        Preprocessed <- Preprocessed%>%select(-c(ADD,EO)) #ID5からはADDとEOを除く        
    }else{
        Preprocessed <- Preprocessed%>%select(-c(EO)) #それ以外の被験者からはEOのみ除く       
    }
    
    flag <- 0 #最適な筋シナジー数をマーク
    list_overall_VAF <- c()
    list_second_min_VAFm <- c()
    list_min_VAFm <- c()
    
    #保存ディレクトリの作成
    path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/Preprocessed_bind/",sep="")
    fname <- file_name%>%sub(path,"",.)
    dname <- fname%>%sub(".csv","",.)
    path_Coef <- path%>%sub("Preprocessed_bind",paste(rand,"/Coef/",dname,sep=""),.)
    path_Synergy <- path%>%sub("Preprocessed_bind",paste(rand,"/Synergy/",dname,sep=""),.)
    path_Coef_graph <- path%>%sub("Preprocessed_bind",paste(rand,"/Coef_graph",sep=""),.)
    path_Synergy_graph <- path%>%sub("Preprocessed_bind",paste(rand,"/Synergy_graph",sep=""),.)
    path_list <- list(path_Coef, path_Synergy, path_Coef_graph, path_Synergy_graph)
    for (path in path_list){
        if (file.exists(path) == FALSE) dir.create(path,recursive=T)    
    }
    
    #NMF解析
    for (i in 1:muscle_num){
        NMF <- nmf(Preprocessed,rank=i)#,seed=1234)#seed=(数字)とすることで，毎回の初期値を(数字)に固定
        Coef <- basis(NMF)
        Synergy <- coef(NMF)
        e <- Preprocessed - (Coef %*% Synergy) #余剰誤差
        e <- as_tibble(e)
        colsum_e_sqr <- transmute_all(e,list(~.x*.x))%>%transmute_all(.,~sum(.x))%>%slice(.,1)#%>%as.numeric(count_data)
        allsum_e_sqr <- colsum_e_sqr%>%sum(.)  
        colsum_Preprocessed_sqr <- transmute_all(Preprocessed,list(~.x*.x))%>%transmute_all(.,~sum(.x))%>%slice(.,1)#%>%as.numeric(count_data)
        allsum_Preprocessed_sqr <- colsum_Preprocessed_sqr%>%sum(.) 
        
        #VAFの計算
        overall_VAF <- 1.0-(allsum_e_sqr/allsum_Preprocessed_sqr)
        muscle_VAF <- 1.0-(colsum_e_sqr/colsum_Preprocessed_sqr)
        sort_muscle_VAF <- sort(muscle_VAF)
        
        # muscle VAFを表示
        min_VAFm <- as.numeric(sort_muscle_VAF[1]) 
        second_min_VAFm <- as.numeric(sort_muscle_VAF[2]) 
        print(paste("Synergy",i,sep=""))
        print("minimam VAF")
        print(sort_muscle_VAF[1])
        print("second minimam VAF")
        print(sort_muscle_VAF[2])
        
#         print(paste("overall_VAF=",overall_VAF, sep=""))
#         print(paste("muscle_VAF(second min)=",second_min_VAFm, sep=""))
#         print(paste("muscle_VAF(min)=",min_VAFm, sep=""))
        
        list_overall_VAF <- append(list_overall_VAF, overall_VAF)
        list_second_min_VAFm <- append(list_second_min_VAFm, second_min_VAFm)
        list_min_VAFm <- append(list_min_VAFm, min_VAFm)
        
        #保存ファイルの名前などの設定
        if(str_detect(fname, pattern="walk")){      #歩行
            if(label==0){ 
                pdf_name <- paste("_rank=",i,"_stance.pdf",sep="")
                fname_Coef <- paste(path_Coef,fname,sep="")%>%sub(".csv", paste("_rank=",i,"_stance.csv",sep=""),.)
                fname_Synergy <- paste(path_Synergy,fname,sep="")%>%sub(".csv", paste("_rank=",i,"_stance.csv",sep=""),.)
                fname_VAF <- paste(path_Synergy,fname,sep="")%>%sub(".csv", paste("_stance_VAF.csv",sep=""),.)
                gname_VAF <- paste(path_Synergy_graph,fname,sep="")%>%sub(".csv", paste("_stance_VAF.pdf",sep=""),.)              
                dname_labeled <- paste(dname,"_stance",sep="")
            }else{
                pdf_name <- paste("_rank=",i,"_swing.pdf",sep="")
                fname_Coef <- paste(path_Coef,fname,sep="")%>%sub(".csv", paste("_rank=",i,"_swing.csv",sep=""),.)
                fname_Synergy <- paste(path_Synergy,fname,sep="")%>%sub(".csv", paste("_rank=",i,"_swing.csv",sep=""),.)
                fname_VAF <- paste(path_Synergy,fname,sep="")%>%sub(".csv", paste("_swing_VAF.csv",sep=""),.)
                gname_VAF <- paste(path_Synergy_graph,fname,sep="")%>%sub(".csv", paste("_swing_VAF.pdf",sep=""),.)
                dname_labeled <- paste(dname,"_swing",sep="")
            }
        }else{                                      #スラックライン
            pdf_name <- paste("_rank=",i,".pdf",sep="")
            fname_Coef <- paste(path_Coef,fname,sep="")%>%sub(".csv", paste("_rank=",i,".csv",sep=""),.)
            fname_Synergy <- paste(path_Synergy,fname,sep="")%>%sub(".csv", paste("_rank=",i,".csv",sep=""),.)
            fname_VAF <- paste(path_Synergy,fname,sep="")%>%sub(".csv","_VAF.csv",.)  
            gname_VAF <- paste(path_Synergy_graph,fname,sep="")%>%sub(".csv","_VAF.pdf",.)   
            dname_labeled <- dname            
        }
        
        #筋シナジーと係数行列のcsv出力
        write_csv(as.data.frame(Synergy),fname_Synergy)
        Coef <- Coef%>%set_colnames(paste("Coef",1:i,sep=""))
        write_csv(as.data.frame(Coef),fname_Coef)
        
        #最適な筋シナジー数を判定
        if ((overall_VAF>0.9)&&(second_min_VAFm>0.75)){
            flag <- flag+1
            print(paste("over criterions! The number of Synergy is ",i,sep=""))
            
            #以下は初めて基準を満たしたシナジーに対する処理
            if (flag==1){
                best_num_synergy <- i
                best_Synergy <- as_tibble(Synergy)   
                
                #筋シナジーのグラフ出力
                gname <- fname%>%sub(".csv",pdf_name,.)
                gname_Synergy <- paste(path_Synergy_graph,gname,sep="")   
#                 print(paste("gname_Synergy:",gname_Synergy,sep=""))
                pdf(gname_Synergy)
                graph_text1 <- paste(id,"/",dname_labeled," ",rand)
                graph_text2 <- paste("overall_VAF=",signif(overall_VAF,digits=4), ", min VAFm=",signif(min_VAFm,digits=4),sep="")
                ifelse(i %% 2 == 0,par(mfrow=c(2,i/2)),par(mfrow=c(2,(i+1)/2)))
                par(oma = c(1.0, 2.0, 0.2, 0)) # 下・左・上・右の順で余白を設定
                if(id=="ID3"){
                    col_list <- c("yellow1","gold","plum1","hotpink","maroon1","deeppink4","cadetblue1","turquoise2","blue3","blue4","chartreuse","green3","forestgreen","gray48")
                }else if(id=="ID5"){
                    col_list <- c("yellow1","gold","plum1","hotpink","maroon1","cadetblue1","turquoise2","darkturquoise","blue3","blue4","chartreuse","green3","forestgreen","gray48")                    
                }else{
                    col_list <- c("yellow1","gold","plum1","hotpink","maroon1","deeppink4","cadetblue1","turquoise2","darkturquoise","blue3","blue4","chartreuse","green3","forestgreen","gray48")
                }
                for (num in 1:i){
                    barplot(Synergy[num,1:muscle_num],main=paste("Synergy",num,sep=""),cex.names=0.6,cex.axis=1, horiz=T, las=1, col=col_list)  
                }
                mtext(graph_text1,
                     outer = TRUE,      # 作図領域の外の余白に書く
                     side = 3,          # 上の余白に書く
                     cex = 1,         # 字の大きさ
                     line = -1.5)        # 外に向かって 3行離れたところに書く
                mtext(graph_text2,
                     outer = TRUE,      # 作図領域の外の余白に書く
                     side = 1,          # 下の余白に書く
                     cex = 1.1,         # 字の大きさ
                     line = -1)        # 外に向かって 1行離れたところに書く    
                dev.off()    

                #係数行列のグラフ出力 
                gname_Coef <- paste(path_Coef_graph,gname,sep="")
                plots_coef <- list()
                for (num in 1:i){
                    time <- 1:nrow(Coef)
                    Coef_data <- data.frame(
                        time = time,
                        Coef = c(Coef[,num])
                    )
                    #スペース足りないのでVAFの値を有効数字4桁に丸める
                    graph_text <- paste(id,"/",fname,"  overall_VAF=",signif(overall_VAF,digits=4),", muscle_VAF(min)=",signif(min_VAFm,digits=4),sep="")
                    ifelse(num==1,
                        plot_title_Coef <- paste("Coef",num,"   ",graph_text,sep=""),
                        plot_title_Coef <- paste("Coef",num,sep=""))
                    Coef_plot <- ggplot(Coef_data,aes(x=time,y=Coef),title=num)+
                                    geom_line()+labs(title=plot_title_Coef)+ 
                                    theme(axis.title=element_text(size=3,face="plain"))
                    plots_coef[[num]] <- Coef_plot
                }
                pdf(gname_Coef)
                multiplot(plotlist=plots_coef,cols=1,graph_text=graph_text)
                dev.off()                 
                
                #歩行の1strideにおける係数行列の平均±SDを計算
                gname_Coef_MEAN_SD <- paste(path_Coef_graph,sub(".pdf","_MEAN_SD.pdf",gname),sep="")
                if(str_detect(gname_Coef_MEAN_SD,pattern="walk")){
#                     print(phase_tlist)
#                     print(phase_tlist_sum)
                    Coef_mean_SD(Coef,phase_tlist,phase_tlist_sum,gname_Coef_MEAN_SD,label,id,rand)
                }
            }
        }
    }

    #VAFの推移グラフ
    VAF_list <- rbind(list_overall_VAF,list_min_VAFm,list_second_min_VAFm)
    colnames(VAF_list) <- paste("Syn",1:muscle_num)
    pdf(gname_VAF)
    plot(1:muscle_num,VAF_list[1,],type="o",ylim=c(0,1),col="red",ann=F) #overall VAF    
    par(new=T)
    plot(1:muscle_num,VAF_list[2,],type="o",ylim=c(0,1),col="blue",ann=F) #second min VAFm
    par(new=T)
    plot(1:muscle_num,VAF_list[3,],type="o",ylim=c(0,1),col="green",xlab="The number of Synergy", ylab="VAF value") #min VAFm                             
    abline(h=0.9,col="deeppink4",lty="dashed")
    abline(h=0.75,col="blue4",lty="dashed")
    labels <- c("Overall VAF","min VAFm","second min VAFm")
    legend("bottomright", legend=labels, col=c("red","blue","green"), pch=c(1,1,1), lty=c(1,1,1))
    title(main=graph_text1)
    dev.off()    
          
    #VAFの推移をcsv出力
    VAF_list <- as.data.frame(VAF_list)
    write_csv(VAF_list,fname_VAF)

    return(list(syn_num=best_num_synergy,Synergy=best_Synergy))
}

In [6]:
id_list <- c("ID1","ID3","ID4","ID5","ID6","ID7","ID8")　#被験者のIDリストを入力
for (id in id_list){　　#各被験者のデータ
    print("enter")
    file_path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/", id,"/Preprocessed_bind", sep="")
    file_list <- list.files(path=file_path, pattern=".csv",recursive=F,full.names=T) #フルパスで取得
    fname_walk <- str_subset(file_list,pattern="walk")
    header_walk <- c("time","y_pos","x_pos","TA","PERL","VL","VM","RF","ADD","LG","MG","SOL","BF","ST","TFL","Gmed","GM","ERSP","EO","phase")

    # 歩行データのNMF解析
    if(id=="ID3"||id=="ID5"){
        muscle_num <- 14
    }else{
        muscle_num <- 15
    }
    
    #ここに各乱数に対応するfor文を入れる
    random_list <- c(paste("Random",1:10,sep=""))
    for (rand in random_list){
        print(rand)
        NMF_walk_stance <- NMF_analysis(header_walk,fname_walk,muscle_num,id,label=0,task="walk",rand) #立脚期
        syn_num_stance <- NMF_walk_stance$syn_num
        Synergy_stance <- NMF_walk_stance$Synergy

        NMF_walk_swing <- NMF_analysis(header_walk,fname_walk,muscle_num,id,label=1,task="walk",rand) #遊脚期
        syn_num_swing <- NMF_walk_swing$syn_num
        Synergy_swing <- NMF_walk_swing$Synergy

        for (file in file_list){
            # スラックラインデータのNMF解析
            if(str_detect(file,pattern="slack")){
                fname_slack <- file
                header_slack <- c("time","ankle_X","ankle_Z","TA","PERL","VL","VM","RF","ADD","LG","MG","SOL","BF","ST","TFL","Gmed","GM","ERSP","EO")
                NMF_slack <- NMF_analysis(header_slack,fname_slack,muscle_num,id,label=-2,task="slack",rand)
                syn_num_slack <- NMF_slack$syn_num
                Synergy_slack <- NMF_slack$Synergy
            }
        }
    }
}

ERROR: Error in parse(text = x, srcfile = src): <text>:1:56: unexpected input
1: id_list <- c("ID1","ID3","ID4","ID5","ID6","ID7","ID8")　
                                                           ^


## 乱数間で最適な筋シナジー同士の相関を求める

In [7]:
#シナジー数が最小 かつ min VAFmが最大となる乱数の番号を取得
min_Synergy <- function(random_list,syn_num,big_task,small_task,col_types){
    #シナジー数が最小となる乱数番号を取得
    min_num <- min(syn_num)
    min_ind <- which(syn_num==min_num)
    min_rand <- random_list[c(min_ind)]

    VAFm_list <- c()
    if(length(min_rand)>1){
        for (rand in min_rand){
            if(big_task=="walk4.5km"){
                file_path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",rand,"/Synergy/",big_task, sep="")
                VAF_flist <- list.files(file_path,pattern="VAF",recursive=F,full.names=T)
                VAF_fname <- str_subset(VAF_flist,pattern=small_task)

            }else if(big_task=="slack"){
                file_path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",rand,"/Synergy/",small_task, sep="")
                VAF_flist <- list.files(file_path,pattern="VAF",recursive=F,full.names=T)
                VAF_fname <- str_subset(VAF_flist,pattern=small_task) 
            }
            VAF <- read_csv(VAF_fname,col_types=col_types)
            min_VAFm <- as.numeric(VAF[2,min_num])
            VAFm_list <- append(VAFm_list,min_VAFm)  # min VAFmを追加       
        }

        # min VAFmが最大となる乱数番号を取得
        max_VAFm <- max(VAFm_list)
        max_VAFm_ind <- which(VAFm_list==max_VAFm)
        if((length(max_VAFm_ind)>1) && (syn_num==1)) max_VAFm_ind <- max_VAFm_ind[1]
        max_VAFm_rand <- min_rand[max_VAFm_ind]
        best_rand <- max_VAFm_rand

    }else{
        best_rand <- min_rand
    }

    return(list(min_num=min_num,best_rand=best_rand))
}

In [8]:
Correlation_best <- function(fname_slack,walk_syn_num,Synergy_walk,slack_syn_num,Synergy_slack,id,phase,walk_best_rand,slack_best_rand){
    
    path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/correlation_best/",sep="")    
    fname_cor <- paste(path,phase,fname_slack,sep="")
    gname_cor <- sub("correlation_best","correlation_best_graph",fname_cor)%>%sub(".csv",".pdf",.)
    cor_path <- paste(path,phase,sep="")
    corg_path <- paste(sub("correlation_best","correlation_best_graph",cor_path),sep="")
    
    dlist <- list(cor_path,corg_path)
    for (dir in dlist){
        if (file.exists(dir) == FALSE) dir.create(dir,recursive=T)        
    }
    
    print(walk_syn_num)
    print(slack_syn_num)
        
    cor_matrix <- matrix(rep(2,slack_syn_num*walk_syn_num),nrow=slack_syn_num,ncol=walk_syn_num) #相関係数を格納するための行列を作成
    for (s_num in 1:slack_syn_num){
        for (w_num in 1:walk_syn_num){
            slack <- t(Synergy_slack[s_num,])
            walk <- t(Synergy_walk[w_num,])
            correlation <- cor(slack,walk)
            cor_matrix[s_num,w_num] <- correlation
        }
    }
    colnames(cor_matrix) <- paste("walk_syn",1:walk_syn_num,sep="")
    rownames(cor_matrix) <- paste("slack_syn",1:slack_syn_num,sep="")

    #相関係数をプロット
    pdf(gname_cor)
    par(oma = c(1.0, 2.0, 0.2, 0)) # 下・左・上・右の順で余白を設定
    cut <- paste("_bind_rank=",slack_syn_num,".csv",sep="")
    title <- fname_slack%>%sub(path,"",.)%>%sub("/","",.)%>%sub(cut,"",.)
    print(title)
    print(cut)
    cor_title <- ifelse(phase=="stance",paste(id,"/",slack_best_rand,"_",title," vs ",walk_best_rand,"_walk_stance",sep=""),
                    paste(id,"/",slack_best_rand,"_",title," vs ",walk_best_rand,"_walk_swing",sep=""))
    col <- gplots::colorpanel(n=256, low="deepskyblue", mid="white", high="violetred")
    cex.before <- par("cex") #テキストのサイズを小さくする
    par(cex = 0.7) 
    corrplot(cor_matrix,method="shade",shade.col=NA,#title=cor_title,
             tl.col="black",tl.srt=45,
             col=col,number.digits=4,addCoef.col="black") #相関係数を有効数字4桁で表示
    mtext(cor_title,
        outer = TRUE,      # 作図領域の外の余白に書く
        side = 3,          # 上の余白に書く
        cex = 1.1,         # 字の大きさ
        line = -3)         # 外に向かって3行離れたところに書く 
    dev.off()    

    #結果を出力
    cor_matrix <- as_tibble(cor_matrix)
    write_csv(cor_matrix,fname_cor)
}

In [9]:
#最適なシナジーのplot
plot_Synergy <- function(fpath,fname,syn_num,best_rand,Synergy,id,muscle_num){
    task <- fname%>%sub(fpath,"",.)%>%sub(".csv","",.)%>%sub("/","",.)
    print(task)
    gpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/Synergy_best_graph/",sep="")
    gname <- paste(gpath,task,".pdf",sep="")
    if (file.exists(gpath) == FALSE) dir.create(gpath,recursive=T)  
                
    pdf(gname)
    graph_text <- paste(id,"/",task," ",best_rand)
    ifelse(syn_num%%2==0, par(mfrow=c(2,syn_num/2)), par(mfrow=c(2,(syn_num+1)/2)))
    par(oma = c(1.0, 2.0, 0.2, 0)) # 下・左・上・右の順で余白を設定
    if(id=="ID3"){
        col_list <- c("yellow1","gold","plum1","hotpink","maroon1","deeppink4","cadetblue1","turquoise2","blue3","blue4","chartreuse","green3","forestgreen","gray48")
    }else if(id=="ID5"){
        col_list <- c("yellow1","gold","plum1","hotpink","maroon1","cadetblue1","turquoise2","darkturquoise","blue3","blue4","chartreuse","green3","forestgreen","gray48")                    
    }else{
        col_list <- c("yellow1","gold","plum1","hotpink","maroon1","deeppink4","cadetblue1","turquoise2","darkturquoise","blue3","blue4","chartreuse","green3","forestgreen","gray48")
    }
    
    for (num in 1:syn_num){
        Syn <- as.numeric(Synergy[num,])
        print(Syn)
        barplot(Syn,main=paste("Synergy",num,sep=""),cex.names=0.6,cex.axis=1, horiz=T, las=1, col=col_list,names.arg=colnames(Synergy))  
    }
    mtext(graph_text,
         outer = TRUE,      # 作図領域の外の余白に書く
         side = 3,          # 上の余白に書く
         cex = 1,         # 字の大きさ
         line = -1.5)        # 外に向かって 3行離れたところに書く
    dev.off()   
}

In [10]:
id_list <- c("ID1","ID3","ID4","ID5","ID6","ID7","ID8")　#被験者のIDリストを入力

for (id in id_list){　　#各被験者のデータ
    print(id)
    if(id=="ID3"||id=="ID5"){
        col_types <- "dddddddddddddd"
        muscle_num <- 14
    }else{
        col_types <- "ddddddddddddddd"
        muscle_num <- 15
    }
           
    #シナジーのグラフが保存されているディレクトリ
    file_path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/Preprocessed_bind", sep="")
    task_list <- list.files(path=file_path, pattern=".csv",recursive=F,full.names=F) #相対パスで取得
    walk_file <- str_subset(task_list,pattern="walk")
    stance_num <- c()
    swing_num <- c()
    walk_gname <- c()
    random_list <- c(paste("Random",1:10,sep=""))
    
    #歩行のシナジー数を取得
    for (rand in random_list){
        #シナジーのグラフが保存されているディレクトリ
        file_path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",rand,"/Synergy_graph", sep="")
        file_list <- list.files(file_path,pattern=".pdf",recursive=F,full.names=F)
        file_list <- str_subset(file_list,pattern="VAF",negate=T)
        walk_gname <- str_subset(file_list,pattern="walk")
        for (file in walk_gname){
            file_csv <- sub("pdf","csv",file)
            read_fname <- paste(sub("Synergy_graph","Synergy",file_path),"/walk4.5km/",file_csv,sep="")
            num <- read_csv(read_fname,col_types=col_types)%>%nrow(.)
            if(str_detect(read_fname,pattern="stance")) stance_num <- append(stance_num,num)
            if(str_detect(read_fname,pattern="swing")) swing_num <- append(swing_num,num)
        }
    }
    
    #歩行の最適なシナジーを決定
    stance <- min_Synergy(random_list,stance_num,"walk4.5km","stance",col_types)
    stance_syn_num <- stance$min_num
    stance_best_rand <- stance$best_rand
    stance_fpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",stance_best_rand,"/Synergy/walk4.5km", sep="")
    stance_file <- list.files(stance_fpath,pattern=paste("rank=",stance_syn_num,"_stance.csv",sep=""),full.names=T)
    stance_Synergy <- read_csv(stance_file,col_types=col_types)
            
    swing <- min_Synergy(random_list,swing_num,"walk4.5km","swing",col_types)
    swing_syn_num <- swing$min_num
    swing_best_rand <- swing$best_rand
    swing_fpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",swing_best_rand,"/Synergy/walk4.5km", sep="")
    swing_file <- list.files(swing_fpath,pattern=paste("rank=",swing_syn_num,"_swing.csv",sep=""),full.names=T)
    swing_Synergy <- read_csv(swing_file,col_types=col_types)    
            
    #歩行の最適なシナジーをプロット
    plot_Synergy(stance_fpath,stance_file,stance_syn_num,stance_best_rand,stance_Synergy,id,muscle_num)
    plot_Synergy(swing_fpath,swing_file,swing_syn_num,swing_best_rand,swing_Synergy,id)
            
    #スラックラインの最適なシナジーを決定，歩行との相関係数を計算
    for (task in task_list){
        slack_num <- c()
        if(str_detect(task,pattern="slack")){
            task <- sub(".csv","",task)
            #スラックラインの最適なシナジーを決定
            for (rand in random_list){
                #シナジーのグラフが保存されているディレクトリ
                file_path <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",rand,"/Synergy_graph", sep="")
                file_list <- list.files(file_path,pattern=".pdf",recursive=F,full.names=F)
                file_list <- str_subset(file_list,pattern="VAF",negate=T)
                slack_gname <- str_subset(file_list,pattern=task)
                slack_fname <- sub("pdf","csv",slack_gname)
                read_fname <- paste(sub("Synergy_graph","Synergy",file_path),"/",task,"/",slack_fname,sep="")
                num <- read_csv(read_fname,col_types=col_types)%>%nrow(.)
                slack_num <- append(slack_num,num)
            }
            #スラックラインの最適なシナジーを決定
            slack <- min_Synergy(random_list,slack_num,"slack",task,col_types)
            slack_syn_num <- slack$min_num
            slack_best_rand <- slack$best_rand
            slack_fpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",slack_best_rand,"/Synergy/",task, sep="")
            slack_file <- list.files(slack_fpath,pattern=paste("rank=",slack_syn_num,".csv",sep=""),full.names=T)
            slack_Synergy <- read_csv(slack_file,col_types=col_types)    
            slack_fname <- sub(slack_fpath,"",slack_file)
            plot_Synergy(slack_fpath,slack_file,slack_syn_num,slack_best_rand,slack_Synergy,id,muscle_num)
            
            #歩行とスラックラインの最適なシナジー同士で相関係数を計算      
            Correlation_best(slack_fname,stance_syn_num,stance_Synergy,slack_syn_num,slack_Synergy,id,"stance",stance_best_rand,slack_best_rand)
            Correlation_best(slack_fname,swing_syn_num,swing_Synergy,slack_syn_num,slack_Synergy,id,"swing",swing_best_rand,slack_best_rand)          
        }
    }
}

ERROR: Error in parse(text = x, srcfile = src): <text>:1:56: unexpected input
1: id_list <- c("ID1","ID3","ID4","ID5","ID6","ID7","ID8")　
                                                           ^


## 歩行（1stride）における係数行列の平均$\pm$標準偏差を保存

In [11]:
library(readr)
library(Rmisc) #tidtverseより先に読み込む
library(tidyverse)
library(signal)
library(magrittr)
library(grid) 
library(NMF)
library(dplyr)
library(ggplot2)
library(cowplot) 
library(corrplot) #相関係数のplotに使用
library(squash)   #plot時の色指定に使用
library(maptools)
library(gplots)

ERROR: Error in library(readr): there is no package called ‘readr’


In [12]:
#labelに該当するphaseの連続回数（1strideのstep数）を求める
get_phase_tlist <- function(fdata_walk_labeled,label){
    fdata_walk_labeled <- as.data.frame(fdata_walk_labeled)
    phase <- fdata_walk_labeled[,ncol(fdata_walk_labeled)]
    count <- 0
    phase_tlist <- c()
    phase_tlist_sum <- c()
    for(i in 1:length(phase)){
        if(phase[i]==-1&&(phase[i]==0||phase[i]==1)&&(phase[i]==0||phase[i]==1)){ #途中(おかしな場所)でlabelが-1であれば，その前までのphase_tlistを返す 
            print("Warning! The data contains invalid label.") 
            break
        }
        if(phase[i]==label){
            count <- count+1
        }else if(i>1&&(phase[i-1]==label)&&(phase[i]!=label)){ #立脚期/または遊脚期が終わってからやっと長さ情報追加＝途中で終わった分は追加されない
            phase_tlist <- append(phase_tlist,count)
            phase_tlist_sum <- cumsum(phase_tlist) #累積和を計算
        }else{
            count <- 0
        }
    }

    return(list(phase_tlist=phase_tlist, phase_tlist_sum=phase_tlist_sum))
}

In [13]:
#1strideにおける係数行列のmean±SDを計算/csv,グラフ出力
Save_Coef_MEAN_SD <- function(task,rand,phase_str,phase_tlist,phase_tlist_sum,label=label){
    #保存するファイルの名前
    fpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",rand,"/Coef/",task,"/",sep="") #係数行列のグラフ名
    gpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/",rand,"/Synergy_graph/",sep="")
    fname <- list.files(gpath,pattern=phase_str,full.names=F)%>%sub(".pdf",".csv",.) 
    fname_Coef <- paste(fpath, fname, sep="")
    Coef <- read_csv(fname_Coef)
    fname_MEAN_SD <- sub(".csv","_MEAN_SD.csv",fname_Coef)     
    
    #保存先の作成
    ifelse(label==0, x_axis <- "stance [%]", x_axis <- "swing [%]") 
    MEAN_SD <- NULL
    
    Syn_num <- ncol(Coef)
    header_list <- c()
    for (i in 1:Syn_num){ #各シナジーに対する処理
        for (j in 2:length(phase_tlist)){ # 各strideに対する処理， 始めのstride（最初の1周期）は無視して計算
            time <- 0:(phase_tlist[j]-1)
            norm_time <- sapply(time,function(x) {return(x/max(time))}) #正規化して周期長を0から1に揃える
            onecycle <- (phase_tlist_sum[j-1]+1):phase_tlist_sum[j]
            onecycle_Coef <- t(Coef[onecycle,i])
            print(length(norm_time))
            print(length(onecycle_Coef))
            sp <- splinefun(norm_time,onecycle_Coef) #関数spを返す 
            xest <- seq(0,1,by=0.01) #新しい時間軸
            yest <- sp(xest) #新しい時間軸に対するCoefの値

            if (j==2){
                Spline_list <- cbind(xest,yest)
            }else{
                Spline_list <- bind_cols(as.data.frame(Spline_list), as.data.frame(yest))
            }
        }
        #平均値, SDを計算    
        header_mean <- paste("Syn",i,"mean",sep="")
        header_pSD <- paste("Syn",i,"mean_psd",sep="")
        header_mSD <- paste("Syn",i,"mean_msd",sep="")        
        header <- c(header_mean, header_pSD, header_mSD)
        header_list <- append(header_list,header)
        Stride <- paste("Stride",1:(length(phase_tlist)-1),sep="")
        colnames(Spline_list) <- append(Stride,"time",after=0)
        mean <- apply(Spline_list[,2:length(phase_tlist)], MARGIN=1, mean) #各行（時間）に対して平均値を計算
        sd <- apply(Spline_list[,2:length(phase_tlist)], MARGIN=1, sd)        
        MEAN_SD <- cbind(MEAN_SD,mean)%>%cbind(.,mean+sd)%>%cbind(.,mean-sd)
    }
    #平均, 平均+SD, 平均-SD
    print(MEAN_SD)    
    MEAN_SD <- MEAN_SD%>%set_colnames(header_list)
    write_csv(as.data.frame(MEAN_SD),fname_MEAN_SD)
}

In [14]:
ID_list <- c("ID1","ID3","ID4","ID5","ID6","ID7","ID8")
Rand_list <- paste("Random",c(5,7,5,10,9,4,9,4,9,5,3,6,3,8),sep="") #最適な筋シナジーの初期値番号（(立脚，遊脚)*各被験者）
fname_Randlist <- "/Users/chika/19-yamane-tidyverse/2001-02/Rand_list.csv"
write_csv(as.data.frame(t(Rand_list)),fname_Randlist)

id_num <- 1
for (id in ID_list){
    #軌道+EMGのtibbleを読み込む
    EMG_fpath <- paste("/Users/chika/19-yamane-tidyverse/2001-02/",id,"/Preprocessed_bind/",sep="")
    fwalk <- list.files(EMG_fpath,pattern="walk")
    EMG_fwalk <- paste(EMG_fpath,fwalk,sep="")
    
    #歩行周期のタイミングを判定
    task <- sub(".csv","",fwalk)
    fdata_walk_labeled <- read_csv(EMG_fwalk,col_types="dddddddddddddddddddi")
    res_stance <- get_phase_tlist(fdata_walk_labeled,label=0) #1strideごとのstep数を取得(立脚期)
    phase_tlist_stance <- res_stance$phase_tlist 
    phase_tlist_sum_stance <- res_stance$phase_tlist_sum
    res_swing <- get_phase_tlist(fdata_walk_labeled,label=1) #1strideごとのstep数を取得（遊脚期）
    phase_tlist_swing <- res_swing$phase_tlist 
    phase_tlist_sum_swing <- res_swing$phase_tlist_sum

    #Coef（平均±SD）のデータ保存
    rand_stance <- Rand_list[2*id_num-1]
    rand_swing <- Rand_list[2*id_num]
    Save_Coef_MEAN_SD(task,rand_stance,"stance.pdf",phase_tlist_stance,phase_tlist_sum_stance,label=0) #立脚期
    Save_Coef_MEAN_SD(task,rand_swing,"swing.pdf",phase_tlist_swing,phase_tlist_sum_swing,label=1) #遊脚期
    
    id_num <- id_num+1
}

ERROR: Error in write_csv(as.data.frame(t(Rand_list)), fname_Randlist): could not find function "write_csv"


### 以下お試し用セル

In [15]:
count <- read.table("kari_data2.txt", sep = "\t", header = T, row.names = 1)
print(count)
count <- as_tibble(count)
count_data <- transmute_all(count,~sum(.x))%>%slice(.,1)%>%as.numeric(count_data)
count_sum <- count_data%>%sum(.)
print(count_sum)
aaa <- sort(count_data)
print(aaa)

Warning message in file(file, "rt"):
“cannot open file 'kari_data2.txt': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [16]:
data <- c(1,2,9,-1)
sortlist <- order(data)
print(sortlist[1])

[1] 4


In [17]:
x <- matrix(1:9, 3, 3)
y <- matrix(1:9, 3, 3)spr0
x %*% y
crossprod(x,y)

ERROR: Error in parse(text = x, srcfile = src): <text>:2:23: unexpected symbol
1: x <- matrix(1:9, 3, 3)
2: y <- matrix(1:9, 3, 3)spr0
                         ^


In [18]:
count <- read.table("kari_data.txt", sep = "\t", header = T, row.names = 1)
count <- as_tibble(count)
count <- as.data.frame(count)
# print(count)
one <- count[,1]
print(one)

Warning message in file(file, "rt"):
“cannot open file 'kari_data.txt': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [19]:
count <- read.table("kari_data2.txt", sep = "\t", header = T, row.names = 1)
print(count)
mcor <- cor(count)
# mcor <- as_tibble(mcor)
# round(mcor,digits=4) 
col1 <- cm.colors
col2 <- gplots::colorpanel(n=256, low="deepskyblue", mid="white", high="violetred")
cex.before <- par("cex")
par(cex = 0.7) #テキストのサイズを小さくする
corrplot(mcor,method="shade",shade.col=NA,
         tl.col="black",tl.srt=45,#type="upper",diag=FALSE,
         col=col1(200),number.digits=4,addCoef.col="black") #相関係数を有効数字4桁で表示
corrplot(mcor,method="shade",shade.col=NA,
         tl.col="black",tl.srt=45,#type="upper",diag=FALSE,
         col=col2,addCoef.col="black")


Warning message in file(file, "rt"):
“cannot open file 'kari_data2.txt': No such file or directory”


ERROR: Error in file(file, "rt"): cannot open the connection


In [20]:
df <- data.frame(x0 = 1:10,
                x1 = 11:20,
                s = sample(c("kosaki", "chitoge", "marika"), 10,
                           replace = TRUE),
                s2 = sample(c("kosaki", "chitoge", NA), 10,
                            replace = TRUE),
                row.names = letters[1:10])
dplyr::filter(df, x0==4)

ERROR: Error in loadNamespace(name): there is no package called ‘dplyr’


In [21]:
NMF_analysis <- function(file_list,muscle_num,label){
    fname_walk <- str_subset(file_list,pattern="walk")
    for (j in 1:length(file_list)){ 　　#各試行のファイル
        if(str_detect(file_list[j],pattern="slack")){
            fname_slack <- file_list[j]
            print(fname_slack)
        }
    }
    return("aaa")
}        

id_list <- list("ID1")　#被験者のIDリストを入力
for (id in id_list){　　　#各被験者のデータ
    print(id)
    file_path <- paste("/Users/chika/19-yamane-tidyverse/1901-02/", id,"/Preprocessed", sep="")
    file_list <- list.files(path=file_path, pattern=".csv",recursive=F,full.names=T) #フルパスで取得
    stance <- NMF_analysis(file_list,muscle_num=14, label=0)
    swing <- NMF_analysis(file_list,muscle_num=14, label=1)
}   

ERROR: Error in parse(text = x, srcfile = src): <text>:3:37: unexpected input
2:     fname_walk <- str_subset(file_list,pattern="walk")
3:     for (j in 1:length(file_list)){ 　
                                       ^


In [22]:
row_names <- paste("Synergy",1:3,sep="")
print(row_names)


[1] "Synergy1" "Synergy2" "Synergy3"


In [23]:
H_list <- list()
matrix1 <- matrix(1:6,nrow=2,ncol=3)
matrix2 <- matrix(7:12,nrow=2,ncol=3)
matrix1 <- as.data.frame(matrix1)
matrix2 <- as.data.frame(matrix2)
H_list <- append(H_list, matrix1)
H_list <- append(H_list, matrix2)
# H_list <- as.data.frame(matrix2)

mat <- matrix(c(1,2,3,4,5,6),nrow=3, ncol=2)
print(mat)
colnames(mat) <- paste("walk_syn",1:2,sep="")
rownames(mat) <- paste("slack_syn",1:3,sep="")
print(mat)


     [,1] [,2]
[1,]    1    4
[2,]    2    5
[3,]    3    6
           walk_syn1 walk_syn2
slack_syn1         1         4
slack_syn2         2         5
slack_syn3         3         6


In [24]:
vec1 <- c(1,3,5)
vec2 <- c(2,4,6)
vec <- rbind(vec1,vec2)
# rownames(vec) <- paste("a",1:2,sep="")
# colnames(vec) <- paste("b",1:3,sep="")
vec <- as.data.frame(vec)
print(mode(vec))

name <- "vec.pdf"
pdf(name)
plot(1:3,vec1,xlim=c(1,3),ylim=c(0,10))
abline(h=2,col="red")
par(new=T)
plot(1:3,vec2,xlim=c(1,3),ylim=c(0,10))
abline(h=4,col="blue")
dev.off()
# vec <- as_tibble(vec)
# print(mode(vec))
# print(vec)
# write_csv(vec,"vec")

[1] "list"


png 
  2

In [25]:
vec <- c(0,0,0,0,0,0,1,1,1,1,1,1,0,0,0)
start <- 10
end <- 45
one_phase <- c(start,end)
vec <- append(vec,one_phase)
print(vec)
vec <- vec[-(length(vec))] #最後の一つの要素除去
print(vec)

 [1]  0  0  0  0  0  0  1  1  1  1  1  1  0  0  0 10 45
 [1]  0  0  0  0  0  0  1  1  1  1  1  1  0  0  0 10


In [26]:
#筋シナジーのmultiplot
multiplot <- function(..., plotlist=NULL, file, cols=1, layout=NULL,graph_text=NULL) {
    plots <- c(list(...), plotlist) # Make a list from the ... arguments and plotlist
    numPlots = length(plots)
    if (is.null(layout)) {  # If layout is NULL, then use 'cols' to determine layout
        layout <- matrix(seq(1, cols * ceiling(numPlots/cols)),
                    ncol = cols, nrow = ceiling(numPlots/cols))
    }
    if (numPlots==1){
        print(plots[[1]])
    }else{
        grid.newpage() # Set up the page
        par(oma = c(0, 0, 4, 0))   # 下・左・上・右の順で余白を設定
        pushViewport(viewport(layout = grid.layout(nrow(layout), ncol(layout))))
        for (i in 1:numPlots) {  # Make each plot, in the correct location
            matchidx <- as.data.frame(which(layout == i, arr.ind = TRUE)) # Get the i,j matrix positions of the regions that contain this subplot
            print(plots[[i]], vp=viewport(layout.pos.row=matchidx$row, layout.pos.col=matchidx$col))
        }
    }       
}

#データフレームからphaseの列を抽出し，ベクトル化
phase <- c(-1,-1,0,0,0,0,0,0,1,1,1,1,1,1,0,0,0,1,1,1,1,1,0,0,0,0,0,1,1,1,1,-1,-1,-1)
label <- 1

count <- 0
#labelに該当するphaseの連続回数（1strideのstep数）を求める
phase_tlist <- c()
phase_tlist_sum <- c()
for(i in 1:length(phase)){
#     if(phase[i]==-1){ #途中でlabelが-1であれば，その前までのphase_tlistを返す
#         print("Warning! The data contains invalid label.") 
#         break
#     }
    if(phase[i]==label){
        count <- count+1
    }else if(i>1&&(phase[i-1]==label)&&(phase[i]!=label)){ #立脚期/または遊脚期が終わってからやっと長さ情報追加＝途中で終わった分は追加されない
        phase_tlist <- append(phase_tlist,count)
        phase_tlist_sum <- cumsum(phase_tlist) #累積和を計算
    }else{
        count <- 0
    }
}
print(phase_tlist)
print(phase_tlist_sum)
#return(phase_tlist=phase_tlist, phase_tlist_sum=phase_tlist_sum)
    
#係数行列の平均±SDをもとめる
plots <- list()
Coef <- matrix(1:(length(phase)*3), nrow=length(phase), ncol=3)
print(Coef)
for (i in 1:ncol(Coef)){ #各シナジーに対する処理
    for (j in 2:length(phase_tlist)){ # 各strideに対する処理， 始めのstride（最初の1周期）は無視して計算
        time <- 0:(phase_tlist[j]-1)
        norm_time <- sapply(time,function(x) {return(x/max(time))}) #正規化して周期長を0から1に揃える
        onecycle <- (phase_tlist_sum[j-1]+1):phase_tlist_sum[j]
        onecycle_Coef <- Coef[onecycle,i]
        sp <- splinefun(norm_time,onecycle_Coef) #関数spを返す 
        xest <- seq(0,1,by=0.01) #新しい時間軸
        yest <- sp(xest) #新しい時間軸に対するCoefの値
        
        if (j==2){
            Spline_list <- cbind(xest,yest)
        }else{
            Spline_list <- bind_cols(as.data.frame(Spline_list), as.data.frame(yest))
        }
    }
    #平均値, SDを計算    
    Stride <- paste("Stride",1:(length(phase_tlist)-1),sep="")
    colnames(Spline_list) <- append(Stride,"time",after=0)
    mean <- apply(Spline_list[,2:length(phase_tlist)], MARGIN=1, mean) #各行（時間）に対して平均値を計算
    sd <- apply(Spline_list[,2:length(phase_tlist)], MARGIN=1, sd)
    
    #平均値,SDのggplot作成
    print(i)
    plot_title_Coef <- paste("Coef",i,sep="")
    Coef_data <- data.frame(stance_phase=xest, mean=mean, sd=sd)
    Coef_plot <- ggplot(data=Coef_data,)+
                    geom_line(aes(x=stance_phase,y=mean))+
                    geom_line(aes(x=stance_phase,y=mean+sd),colour="gray57")+
                    geom_line(aes(x=stance_phase,y=mean-sd),colour="gray57")+
                    xlab("Stance phase") + ylab("Activity of Synergy")+
                    labs(title=plot_title_Coef)+ 
                    theme(axis.title=element_text(size=3,face="plain"))+
                    theme_minimal() #背景の色変更
    plots[[i]] <- Coef_plot
}
    
#平均値，SDのmultiplot
gname_Coef <- "Coef.pdf"
pdf(gname_Coef)
multiplot(plotlist=plots,cols=2)
dev.off() 
    
print(nrow(Coef))
print(length(phase))

[1] 6 5 4
[1]  6 11 15
      [,1] [,2] [,3]
 [1,]    1   35   69
 [2,]    2   36   70
 [3,]    3   37   71
 [4,]    4   38   72
 [5,]    5   39   73
 [6,]    6   40   74
 [7,]    7   41   75
 [8,]    8   42   76
 [9,]    9   43   77
[10,]   10   44   78
[11,]   11   45   79
[12,]   12   46   80
[13,]   13   47   81
[14,]   14   48   82
[15,]   15   49   83
[16,]   16   50   84
[17,]   17   51   85
[18,]   18   52   86
[19,]   19   53   87
[20,]   20   54   88
[21,]   21   55   89
[22,]   22   56   90
[23,]   23   57   91
[24,]   24   58   92
[25,]   25   59   93
[26,]   26   60   94
[27,]   27   61   95
[28,]   28   62   96
[29,]   29   63   97
[30,]   30   64   98
[31,]   31   65   99
[32,]   32   66  100
[33,]   33   67  101
[34,]   34   68  102


ERROR: Error in bind_cols(as.data.frame(Spline_list), as.data.frame(yest)): could not find function "bind_cols"


In [ ]:
a <- c(1,2,3,4) 
a[c(-1:-2,-4)]
length(a)

In [ ]:
random <- c(paste("Random",1:10,sep=""))
print(random)

In [ ]:
ceiling(12.5)

In [27]:

a <- sub("b","",a)
print(a)

ERROR: Error in sub("b", "", a): object 'a' not found


In [28]:
a <- "aaa_rank=4"
#separ以降の文字列を削除
splitLeft<-function(str,separ){
        strsplit(str,separ)[[1]][[1]]
}
b <- splitLeft(a,"rank")
print(b)

[1] "aaa_"


In [29]:
a <- c("aaa","bbb")
d <- c()%>%a
print(d)

ERROR: Error in c() %>% a: could not find function "%>%"


In [30]:
a <- c(5,6,6,7,5,5,5,5,5,5)
min <- min(a)
min <- which(a==min)
print(min)

[1]  1  5  6  7  8  9 10


In [31]:
mean <- c(1,3,4,5)
sd <- c(0.2,0.4,0.3,0.1)
MEAN_SD <- NULL
MEAN_SD <- cbind(MEAN_SD,mean)%>%cbind(.,mean+sd)%>%cbind(.,mean-sd)
print(MEAN_SD)

ERROR: Error in cbind(MEAN_SD, mean) %>% cbind(., mean + sd) %>% cbind(., mean - : could not find function "%>%"


In [32]:
li <- c()
for (i in 1:10){
    a <- paste("Stance_rand",i,sep="")
    b <- paste("Swing_rand",i,sep="")
    li <- append(li,a)%>%append(.,b)
}    
print(li)

ERROR: Error in append(li, a) %>% append(., b): could not find function "%>%"
